### Test
Testing the noteboook

In [ ]:
%matplotlib inline

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.affinity import affine_transform

In [ ]:
# Replace 'path/to/your_shapefile.shp' with the actual path to your shapefile
geojson_path = '2024-01-24_siedlce_2180.geojson'

# Siedlce OSM: '2024-01-24_siedlce_2180.geojson'
# Siedlce ELEMENTY SIECIOWE: '2024-01-24_elem-siec_isa.geojson'
# Siedlce EIK: '2024-01-31_siedlce_EIK_2180.geojson'
# Warka OSM: '2024-01-31_warka_osm_2180.geojson'
#Skierniewice:


In [ ]:
# Read the shapefile
gdf = gpd.read_file(geojson_path)

In [ ]:
# Explore the GeoDataFrame
print(gdf.head())

In [ ]:
# Visualize the data
gdf.plot(figsize=(15, 15))

In [ ]:
rotate_angle = 38

#Siedlce: 38

In [ ]:
df_rotated2 = gdf.rotate(rotate_angle, origin=gdf.unary_union.centroid)

In [ ]:
df_rotated2.plot(figsize=(20, 20))

In [ ]:
# Explore the GeoDataFrame
print(gdf.head())

In [ ]:
# Explore the GeoDataFrame
print(df_rotated2.head())

In [ ]:
# TO DELETE

# Save the GeoDataFrame as a GeoJSON file
df_rotated2.to_file('temp.geojson', driver='GeoJSON')

In [ ]:
#TO DELETE
geojson_path2 = 'temp.geojson'

In [ ]:
# Wczytaj plik GeoJSON
gdf = gpd.read_file(geojson_path2)

In [ ]:
# Oblicz minimalne współrzędne (xmin, ymin) dla każdego z geometrii
min_coords = gdf.bounds.min()

In [ ]:
# Przesuń wszystkie geometrie w lewo i w dół o wartości minimalnych współrzędnych
gdf['geometry'] = gdf.translate(xoff=-min_coords['minx'], yoff=-min_coords['miny'])

In [ ]:
# Explore the GeoDataFrame
print(gdf.head())

In [ ]:
gdf.plot(figsize=(20, 20))

In [ ]:
# Function to scale the x-coordinates by a factor
def scale_x(geometry, factor):
    return affine_transform(geometry, [factor, 0, 0, 1, 0, 0])

In [ ]:
# Apply the scaling operation to the geometry column
gdf['geometry'] = gdf['geometry'].apply(scale_x, factor=1/4)

In [ ]:
# Explore the GeoDataFrame
print(gdf.head())

In [ ]:
df_rotated3 = gdf.rotate(-5, origin=gdf.unary_union.centroid)

In [ ]:
df_rotated3.plot(figsize=(20, 20))

In [ ]:
print(df_rotated3.head())

In [ ]:
# Save the GeoDataFrame as a GeoJSON file
df_rotated3.to_file('2024-01-24_elem-siec_isa_rotated.geojson', driver='GeoJSON')

In [ ]:
import geopandas as gpd
from shapely.geometry import mapping
import xml.etree.ElementTree as ET
from xml.dom import minidom


# RailML XML namespace
railml_ns = {"railml": "http://www.railml.org/schemas/2013"}

# Create RailML XML document
railml_root = ET.Element("railml", nsmap=railml_ns)

# Iterate through GeoDataFrame features
for index, row in gdf.iterrows():
    # Extract geometry from GeoDataFrame
    geometry = row['geometry']

    # Convert Shapely geometry to GeoJSON-like dictionary
    geometry_dict = mapping(geometry)

    # Create RailML track element
    track_element = ET.SubElement(railml_root, "track", nsmap=railml_ns)

    # Create RailML geometry element
    geometry_element = ET.SubElement(track_element, "geometry", nsmap=railml_ns)

    # Add RailML point elements based on GeoJSON-like geometry
    for coord in geometry_dict['coordinates']:
        point_element = ET.SubElement(geometry_element, "point", nsmap=railml_ns)
        point_element.set("x", str(coord[0]))
        point_element.set("y", str(coord[1]))
        # Add additional attributes as needed

# Create a formatted XML string
xml_string = ET.tostring(railml_root, encoding='utf-8')
xml_pretty_string = minidom.parseString(xml_string).toprettyxml(indent="  ")

# Save or print the RailML XML
with open("output_railml.xml", "w") as xml_file:
    xml_file.write(xml_pretty_string)


In [ ]:
# Save the RailML XML to a file
output_file_path = "output_railml.xml"
with open(output_file_path, "w") as xml_file:
    xml_file.write(xml_pretty_string)

# Print a message indicating where the XML is saved
print(f"RailML XML saved to: {output_file_path}")